In [5]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.regularizers import l2
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, \
    Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from convnetskeras.customlayers import convolution2Dgroup, crosschannelnormalization, \
    splittensor, Softmax4D
from customlayers import LRN2D
from keras import regularizers
# global constants
NB_CLASS = 1000         # number of classes
LEARNING_RATE = 0.01
MOMENTUM = 0.9
GAMMA = 0.1
DROPOUT = 0.5
WEIGHT_DECAY = 0.0005   # L2 regularization factor
USE_BN = True           # whether to use batch normalization
# Theano - 'th' (channels, width, height)
# Tensorflow - 'tf' (width, height, channels)
DIM_ORDERING = 'th'
def conv2D_bn(x, nb_filter, nb_row, nb_col,
              border_mode='same', subsample=(1, 1),
              activation='relu', batch_norm=USE_BN,
              weight_decay=WEIGHT_DECAY, dim_ordering=DIM_ORDERING):
    '''

        Info:
            Function taken from the Inceptionv3.py script keras github


            Utility function to apply to a tensor a module conv + BN
            with optional weight decay (L2 weight regularization).
    '''
    if weight_decay:
        W_regularizer = regularizers.l2(weight_decay)
        b_regularizer = regularizers.l2(weight_decay)
    else:
        W_regularizer = None
        b_regularizer = None

    x = Convolution2D(nb_filter, nb_row, nb_col,
                      subsample=subsample,
                      activation=activation,
                      border_mode=border_mode,
                      W_regularizer=W_regularizer,
                      b_regularizer=b_regularizer,
                      dim_ordering=dim_ordering)(x)
    x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)

    if batch_norm:
        x = LRN2D()(x)
        x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)

    return x

def alexnet2(weights_path=None, nb_class=None):

    inputs = Input(shape=(3,227,227))

    conv_1 = Convolution2D(96, 11, 11,subsample=(4,4),activation='relu',
                           name='conv_1')(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2,2))(conv_2)

    x1 = conv2D_bn(conv_2, 128, 5, 5, subsample=(1, 1), border_mode='same')
    y1 = conv2D_bn(conv_2, 128, 5, 5, subsample=(1, 1), border_mode='same')

    conv_2 = merge([x1,y1], mode='concat',concat_axis=1,name="conv_2")

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1,1))(conv_3)
    conv_3 = Convolution2D(384,3,3,activation='relu',name='conv_3')(conv_3)

    conv_4 = ZeroPadding2D((1,1))(conv_3)

    x2 = conv2D_bn(conv_4, 192, 3, 3, subsample=(1, 1), border_mode='same')
    y2 = conv2D_bn(conv_4, 192, 3, 3, subsample=(1, 1), border_mode='same')

    conv_4 = merge([x2,y2], mode='concat',concat_axis=1,name="conv_4")

    conv_5 = ZeroPadding2D((1,1))(conv_4)

    x3 = conv2D_bn(conv_5, 192, 3, 3, subsample=(1, 1), border_mode='same')
    y3 = conv2D_bn(conv_5, 192, 3, 3, subsample=(1, 1), border_mode='same')

    conv_5 = merge([x3,y3], mode='concat',concat_axis=1,name="conv_5")

    conv_5 = MaxPooling2D((3, 3), strides=(2,2),name="convpool_5")(conv_5)

    dense_1 = Flatten(name="flatten")(conv_5)
    dense_1 = Dense(4096, activation='relu',name='dense_1')(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu',name='dense_2')(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    dense_3 = Dense(nb_class,name='dense_3')(dense_3)
    prediction = Activation("softmax",name="softmax")(dense_3)


    base_model = Model(input=inputs, output=prediction)

    if weights_path:
        base_model.load_weights(weights_path)

    return base_model

def load_model(weights_path):

    model = alexnet2(weights_path,100)
    model.compile(optimizer="sgd", loss='categorical_crossentropy')
    return model

model = load_model('../dataset/alexnet_weights.h5')
for l in model.layers:
    if "convolution2d"in l.name:
        print l.name + " - " + str(l.get_weights()[0].shape)
    else:
        print l.name

input_2
conv_1
maxpooling2d_3
convpool_1
zeropadding2d_17
convolution2d_7 - (128, 48, 5, 5)
convolution2d_8 - (128, 48, 5, 5)
zeropadding2d_18
zeropadding2d_20
lrn2d_7
lrn2d_8
zeropadding2d_19
zeropadding2d_21
conv_2
maxpooling2d_4
lambda_2
zeropadding2d_22
conv_3
zeropadding2d_23
convolution2d_9 - (192, 192, 3, 3)
convolution2d_10 - (192, 192, 3, 3)
zeropadding2d_24
zeropadding2d_26
lrn2d_9
lrn2d_10
zeropadding2d_25
zeropadding2d_27
conv_4
zeropadding2d_28
convolution2d_11 - (128, 192, 3, 3)
convolution2d_12 - (128, 192, 3, 3)
zeropadding2d_29
zeropadding2d_31
lrn2d_11
lrn2d_12
zeropadding2d_30
zeropadding2d_32
conv_5
convpool_5
flatten
dense_1
dropout_3
dense_2
dropout_4
dense_3
softmax


In [6]:
import cv2
import keras.backend as K

def deconv(model, target_layer, feat_map, im):
    dlayers = {}
    for l in model.layers:
        dlayers[l.name] = l
    layerNames = [l.name for l in model.layers]

    im = im.reshape((1,) + im.shape)
    symX = K.T.tensor4('x')
    ##%%%%%%%%%%%%%%%%%%%%%%#
    # forward pass
    ##%%%%%%%%%%%%%%%%%%%%%%#
    X_foward = im
    d_switch = {}
    layer_index = layerNames.index(target_layer)

    for lname in layerNames[:layer_index+1]:
        # print('forwarding {} ...'.format(lname))
        T_in, T_out = dlayers[lname].input, dlayers[lname].output

        forward = K.function([T_in], T_out)
        X_foward = forward([X_foward])
        # Record the switch variables
        if "convolution2d" in lname:
            d_switch[lname] = np.where(X_foward <= 0)

    ##%%%%%%%%%%%%%%%%%%%%%%#
    # backward pass
    ##%%%%%%%%%%%%%%%%%%%%%%#
    X_outlayer = X_foward
    # print "*************Deconvolution*************"
    # print "Deconvolving %s ..." % target_layer
    if "maxpooling2d" in target_layer:
        print 'you wanna to see maximal activations in pooling layers?'

    elif "convolution2d" in target_layer:

        output_width, output_height = dlayers[target_layer].output_shape[-2:]
        filter_width, filter_height = dlayers[target_layer].W_shape[2], dlayers[target_layer].W_shape[3]

        # Compute padding needed
        input_width, input_height = X_outlayer.shape[-2:]
        pad_width = (output_width - input_width + filter_width - 1) / 2
        pad_height = (output_height - input_height + filter_height - 1) / 2
        assert isinstance(pad_width, int), "Pad width size issue at layer %s" % lname
        assert isinstance(pad_height, int), "Pad height size issue at layer %s" % lname
        
        # using switch which recorded from forward pass
        X_outlayer[d_switch[target_layer]] = 0
        activation = dlayers[target_layer].activation
        # relu
        X_outlayer = activation(X_outlayer)

        # For the first output layer, choose only maxima activation (point) on the feature map
        ## Setting other feature maps to zero
        list_feat_map = np.arange(X_outlayer.shape[1]).tolist()
        list_feat_map.remove(feat_map)
        X_outlayer[:, list_feat_map, :, :] = 0

        ## Setting other points to zero
        for i in range(X_outlayer.shape[0]):
            iw, ih = np.unravel_index(X_outlayer[i, feat_map, :, :].argmax(), X_outlayer[i, feat_map, :, :].shape)
            maxActivationValue = np.max(X_outlayer[i, feat_map, :, :])
            X_outlayer[i, feat_map, :, :] = 0
            X_outlayer[i, feat_map, iw, ih] = maxActivationValue

        W = dlayers[target_layer].W
        W = W.transpose([1,0,2,3])
        W = W[:,:, ::-1, ::-1]

        conv_out = K.T.nnet.conv2d(input=symX, filters=W, border_mode='valid')
        pad = K.function([symX], K.spatial_2d_padding(symX, padding=(pad_width, pad_height), dim_ordering="th"))
        X_pad = pad([X_outlayer])

        deconv_func = K.function([symX], conv_out)
        X_deconv = deconv_func([X_pad])
        assert X_deconv.shape[-2:]==(output_width, output_height), "Deconv output at {} has wrong size".format(target_layer)

    else:
        raise ValueError('Invalid layer name {}'.format(target_layer))       

    # Iterate remining layers until to input
    X_outlayer = X_deconv
    for lname in layerNames[:layer_index][::-1]:
        # print "Deconvolving {} ...".format(lname)
        # Unpool
        if "maxpooling2d" in lname:
            p1, p2 = dlayers[lname].pool_size
            uppool = K.function([symX], K.resize_images(symX, p1, p2, "th"))
            X_outlayer = uppool([X_outlayer])
        # deconvolution
        elif "convolution2d" in lname:
            output_width, output_height = dlayers[lname].output_shape[-2:]
            filter_width = dlayers[lname].W_shape[2]
            filter_height = dlayers[lname].W_shape[3]

            # Compute padding needed
            input_width, input_height = X_outlayer.shape[-2:]
            pad_width = (output_width - input_width + filter_width - 1) / 2
            pad_height = (output_height - input_height + filter_height - 1) / 2
            assert isinstance(pad_width, int), "Pad width size issue at layer %s" % lname
            assert isinstance(pad_height, int), "Pad height size issue at layer %s" % lname

            X_outlayer[d_switch[lname]] = 0
            activation = dlayers[lname].activation
            X_outlayer = activation(X_outlayer)

            W = dlayers[lname].W
            W = W.transpose([1,0,2,3])
            W = W[:,:, ::-1, ::-1]

            conv_out = K.T.nnet.conv2d(input=symX, filters=W, border_mode='valid')
            pad = K.function([symX], K.spatial_2d_padding(symX, padding=(pad_width, pad_height), dim_ordering="th"))
            X_pad = pad([X_outlayer])

            deconv_func = K.function([symX], conv_out)
            X_deconv = deconv_func([X_pad])
            assert X_deconv.shape[-2:]==(output_width, output_height), "Deconv output at {} has wrong size".format(lname)
            X_outlayer = X_deconv
        elif "padding" in lname:
                pass
        else:
            raise ValueError("Invalid layer name: {}".format(lname))
            
    return X_outlayer

def loadImage(path):
    im = cv2.resize(cv2.imread(imPath), (224, 224)).astype(np.float32)
    imOri = im.copy()
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    return im, imOri

def normalize(arg):
    arg = arg[0]
    minValue = arg.min()
    maxValue = arg.max()
    imNorm = (arg - minValue) / (maxValue-minValue)
    return imNorm.transpose((1,2,0))

In [18]:
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# choose 4 feature maps to visualize
feat_dict = OrderedDict()
feat_dict["zeropadding2d_17"] = 1
feat_dict["zeropadding2d_17"] = 1

# Load data

imPath = '../dataset/food10/AisKacang/AisKacang (3).jpg'
im, imOri = loadImage(imPath)

for k,v in feat_dict.items():
    target_layer = k
    feat_map = v
    output = deconv(model, target_layer, feat_map, im)

    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(imOri/255.)
    plt.subplot(1,2,2)
    plt.imshow(normalize(output))
    plt.title(target_layer+" - "+str(feat_map))
    plt.show()



ValueError: Invalid layer name zeropadding2d_17